In [1]:
## Preamble: Package Loading
import numpy as np
import ipywidgets as ipw
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools as iter
import os
import datetime as dt
import json
import re
import kernel as kr
import regex
# Preamble working directory retreival

<h2> Function Definitions </h2>

The following set of functions are necessary to for the display and comparision of generated results. Scroll down for those results.

<h3> Function Definitions: Data Loading Function </h3>

In [2]:
def psc_load(input_filename):
    output_folder = '/Users/ericpenner/Google_Drive/Research/pan_sel_cntrl/output/'
    input_file_full = ''.join([output_folder,input_filename])
    with open(input_file_full) as f_obj: 
        pscdata = json.load(f_obj)
    inpt_dic = pscdata[0]
    bt_rs = pscdata[1]
    bt_df = pd.DataFrame(bt_rs['beta_cntrd'] ).T
    bt_df.columns = bt_rs['beta_sum_row']
    bt_sm = pd.DataFrame(bt_rs['beta_sum_dat']).T
    bt_sm.columns = bt_rs['beta_sum_row']
    bt_sm.index = bt_rs['beta_sum_clmn']
    al_rs = pscdata[2]
    al_ldf = [pd.DataFrame(al_rs['alpha_cntrd'][j]).T  for j in range(len(al_rs['alpha_cntrd']))]
    #al_stupid = [ al_rs['alpha_sum_row'][:13] , al_rs['alpha_sum_row'][13] , al_rs['alpha_sum_row'][14]]
    al_sm = [pd.DataFrame(al_rs['alpha_sum_dat'][j]).T  for j in range(len(al_rs['alpha_sum_dat']))]
    for j in range(len(al_rs['alpha_sum_dat'])):
        #al_ldf[j].columns = al_stupid[j]
        al_ldf[j].columns = al_rs['alpha_sum_row'][j]
        #al_sm[j].columns = al_stupid[j] 
        al_sm[j].columns = al_rs['alpha_sum_row'][j]
        al_sm[j].index= al_rs['alpha_sum_clmn']

    out = [inpt_dic, bt_df, bt_sm, al_ldf, al_sm]
    return out

<h3>  Function Definitions: DGP Summary Dictionaries Loading Function </h3>

In [3]:
def pscsum_load(input_filename):
    data_folder = '/Users/ericpenner/Google_Drive/Research/pan_sel_cntrl/data/'
    data_file_full = ''.join([data_folder,input_filename])
    with open(data_file_full) as f_obj: 
        pscdata = json.load(f_obj)
    inpt_dict = pscdata[0]
    pcoeff_nms = ([ ''.join(['$\\beta_{',str(1),',',str(i+1),'}$']) for i in range(pscdata[0]['n_end'])]
                  + [''.join(['$\\beta_{',str(2),',',str(i+1),'}$']) for i in range(pscdata[0]['n_exo'])])
    pcoeff = pd.DataFrame(pscdata[1]['pcoeff']).T
    pcoeff.columns = pcoeff_nms
    
    coeff_nms = [([ ''.join(['$\\alpha_{',str(j+1),str(1),',',str(i+1),'}$']) for i in range(pscdata[0]['n_exo'])]
                  + [''.join(['$\\alpha_{',str(j+1),str(2),',',str(i+1),'}$']) for i in range(pscdata[0]['t_inst'])]) 
                 for j in range(pscdata[0]['n_end'])]
    coeff = [pd.DataFrame(pscdata[1]['coeff'][i],columns = coeff_nms[i]) for i in range(len(pscdata[1]['coeff']))]
    return [inpt_dict , pcoeff , coeff ]

<h3>  Function Definitions: Density Plotting Function </h3>

In [5]:
## Plotting Function
def coeffden(coeff,line_nms,x_lm,y_lm,c_h,w,n_eqn,s_eqn):
    nds = len(coeff)
    w = w-1
    s_eqn = s_eqn-1
    if n_eqn > 1:
        coeff = [coeff[i][s_eqn] for i in range(nds)]
    ncfs = coeff[0].shape[0]
    plt.close('all')
    a = [ np.sort(coeff[i].iloc[:,w].values,axis = 0) for i in range(nds) ] 
    h = [ c_h*ncfs**(-1/5)*np.std(a[i]) for i in range(nds)]
    aden = [ kr.mvden(a[i],a[i],h[i],9).reshape(ncfs,1) for i in range(nds) ]
    f,ax = plt.subplots()
    f.set_figheight(7)
    f.set_figwidth(15)
    ax.set_xlim((x_lm[0],x_lm[1]))
    ax.set_ylim((0,y_lm))
    for i in range(nds):
        ax.plot(a[i],aden[i])
    ax.legend(line_nms)
    ax.grid(which = 'both')
    ax.set_title(''.join(['Distribution of Estimated ',coeff[0].columns[w]]))
    plt.show()

<h3>  Function Definitions: Interactive Plot and Tables Display Function </h3>

In [6]:
def tbl_dsp(tables,n_eqn,s_eqn,line_nms):
    s_eqn = s_eqn-1
    if n_eqn > 1:
        for j in range(len(tables)):
            display(tables[j][s_eqn])
            display(''.join(['Case ', str(j+1),':', line_nms[j]]))
    elif n_eqn == 1:
        for j in range(len(tables)):
            display(tables[j])
            display(''.join(['Case ', str(j+1),':', line_nms[j]]))

def cfs_dsp(coeff,tables,n_eqn,y_lm):
    
    box_hlayout = ipw.Layout(display='flex',flex_flow='row',align_items='stretch'
                             ,width='95%')
    
    if n_eqn == 1:
        coeff_sel = ipw.IntSlider( min = 1 , max = coeff[0].shape[1], value = 1, step = 1
                             , description = 'Coefficient:'
                              ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})
    elif n_eqn > 1: 
        coeff_sel = ipw.IntSlider( min = 1 , max = coeff[0][0].shape[1], value = 1, step = 1
                              , description = 'Coefficient:'
                              ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})

    eqn_sel = ipw.IntSlider( min = 1 , max = n_eqn, value = 1, step = 1, description = 'Equation:'
                              ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})

    ch_sel = ipw.FloatSlider(min = 0.1 , max = 3, value = 2.5, step =0.2, description = 'Bandwidth Constant'
                             ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})

    xlim_sel = ipw.FloatRangeSlider( value=[-0.4, 0.4], min=-0.5,max=0.5, step=0.05,description='x limits:'
                                    ,disabled=False ,continuous_update=False, orientation='horizontal'
                                    ,readout=True ,readout_format='.1f', width = 'auto'
                                    ,layout=box_hlayout, style = {'description_width': 'initial'}) 

    coeff_out =  ipw.interactive_output(coeffden ,{'coeff': ipw.fixed(coeff), 'line_nms': ipw.fixed(line_nms)
                                                 ,'x_lm': xlim_sel, 'y_lm': ipw.fixed(y_lm)
                                                 ,'c_h': ch_sel,'w': coeff_sel,'n_eqn': ipw.fixed(n_eqn)
                                                 ,'s_eqn': eqn_sel})  

    table_out = ipw.interactive_output(tbl_dsp,{'tables': ipw.fixed(tables),'n_eqn': ipw.fixed(n_eqn)
                                                ,'s_eqn': eqn_sel,'line_nms': ipw.fixed(line_nms)} )

    return ipw.VBox([table_out,coeff_out,coeff_sel,ch_sel,xlim_sel,eqn_sel])

<h3>  Function Definitions: Input Dictionary Display Function </h3>

In [17]:
def indict_dsp(inpt,depth):
    w_sel = ipw.IntSlider( min = 1 , max = len(inpt), value = 1, step = 1
                                 , description = 'Results Dataset: '
                                  ,width = 'auto', style = {'description_width': 'initial'})
    d_sel = ipw.IntSlider( min = 1 , max = depth , value = 1, step = 1
                                 , description = 'Equation: '
                                  ,width = 'auto', style = {'description_width': 'initial'})
    def in_dcts_dsp(in_dcts,depth,w,d):
        if depth == 1:
            display(in_dcts[w-1])
        elif depth > 1:
            display(in_dcts[w-1][d-1])
            
    dict_out = ipw.interactive_output(in_dcts_dsp,{'in_dcts': ipw.fixed(inpt),'w': w_sel })
    
    if depth == 1:
        dict_out = ipw.interactive_output(in_dcts_dsp,{'in_dcts': ipw.fixed(inpt),'w': w_sel 
                                                       ,'depth': ipw.fixed(depth),'d': ipw.fixed(1)})
        bx = ipw.VBox([w_sel,dict_out])
    elif depth > 1:
        dict_out = ipw.interactive_output(in_dcts_dsp,{'in_dcts': ipw.fixed(inpt),'w': w_sel 
                                                       ,'depth': ipw.fixed(depth),'d': d_sel})
        bx =ipw.VBox([d_sel,w_sel,dict_out])
        
    return bx

<h2>Trial Set 1: Varying Number of Time Periods </h2> 

In [28]:
inpt_filenames0 = ['pscout_6_12_1954.json' ,'pscout_6_12_1220.json' , 'pscout_6_12_1799.json']
line_nms0 = ['n=30', 'n=50' ,'n=70']

res_out0 = [psc_load(inpt_filenames0[i]) for i in range(len(inpt_filenames0))]
estin_dcts0 = [res_out0[i][0] for i in range(len(inpt_filenames0))]
dgp_sum_filenames0 = [ estin_dcts0[i]['input_filename'].replace('pscdata','pscsum')
                      for i in range(len(inpt_filenames0))]
dgp_dicts0 = [pscsum_load(dgp_sum_filenames0[i]) 
             for i in range(len(dgp_sum_filenames0))]
dgpin_dcts0 =  [dgp_dicts0[i][0] for i in range(len(inpt_filenames0))]
true_bcoeffs0 = [dgp_dicts0[i][1] for i in range(len(inpt_filenames0))]
true_acoeffs0 = [dgp_dicts0[i][2] for i in range(len(inpt_filenames0))]
bcoeff0  = [res_out0[i][1] for i in range(len(inpt_filenames0))]
acoeff0  = [res_out0[i][3] for i in range(len(inpt_filenames0))]
btables0 = [res_out0[i][2] for i in range(len(inpt_filenames0))]
atables0 = [res_out0[i][4] for i in range(len(inpt_filenames0))]

<h3> Trial Set 1: DGP Function Input Dictionary Comparison </h3> 

In [29]:
indict_dsp(dgpin_dcts0,1 )

<h3> Trial Set 1: Estimator Function Input Dictionary Comparison </h3>

In [35]:
indict_dsp(estin_dcts0,1)

<h3> Trial Set 1: True Primary Equations Coefficients Comparison </h3>

In [31]:
indict_dsp(true_bcoeffs0,1)

<h3> Trial Set 1: True Secondary Equation Coefficients Comparison </h3> 

In [32]:
indict_dsp(true_acoeffs0,2)

<h3> Trial Set 1: Primary Function Coefficient Estimates </h3>

In [33]:
display(cfs_dsp(bcoeff0,btables0,1,12))

<h3> Trial Set 1: Secondary Function Coefficient Estimates </h3>

In [34]:
display(cfs_dsp(acoeff0,atables0,2,5))

<h3> Trial Set 2: Varying the number of crossections </h3>

In [39]:
inpt_filenames1 = ['pscout_6_12_1220.json' ,'pscout_6_13_1914.json','pscout_6_13_1498.json','pscout_6_13_1227.json' ]
line_nms1 = ['ncr = 5','ncr = 10', 'ncr = 15', 'ncr = 20']

res_out1 = [psc_load(inpt_filenames1[i]) for i in range(len(inpt_filenames1))]
estin_dcts1 = [res_out1[i][0] for i in range(len(inpt_filenames1))]
dgp_sum_filenames1 = [ estin_dcts1[i]['input_filename'].replace('pscdata','pscsum')
                      for i in range(len(inpt_filenames1))]
dgp_dicts1 = [pscsum_load(dgp_sum_filenames1[i]) 
             for i in range(len(dgp_sum_filenames1))]
dgpin_dcts1 =  [dgp_dicts1[i][0] for i in range(len(inpt_filenames1))]
true_bcoeffs0 = [dgp_dicts1[i][1] for i in range(len(inpt_filenames1))]
true_acoeffs0 = [dgp_dicts1[i][2] for i in range(len(inpt_filenames1))]
bcoeff0  = [res_out1[i][1] for i in range(len(inpt_filenames1))]
acoeff0  = [res_out1[i][3] for i in range(len(inpt_filenames1))]
btables0 = [res_out1[i][2] for i in range(len(inpt_filenames1))]
atables0 = [res_out1[i][4] for i in range(len(inpt_filenames1))]

<h3> Trial Set 2: DGP Function Input Dictionary Comparison </h3> 

<h3> Trial Set 2: Estimator Function Input Dictionary Comparison </h3>

In [ ]:
display(cfs_dsp(bcoeff,btables,1,12))
display(cfs_dsp(acoeff,atables,2,5))

In [ ]:
sum_data

In [ ]:
sum_filename = input_filename.replace('pscdata','pscsum')
sum_filename

In [ ]:
sum_data